In [50]:
import pandas as pd

#cargar archivo
data = pd.read_excel ("default.xlsx", header=0)

print (data.head(20))

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


    Unnamed: 0 default student      balance        income
0            1      No      No   729.526495  44361.625074
1            2      No     Yes   817.180407  12106.134700
2            3      No      No  1073.549164  31767.138947
3            4      No      No   529.250605  35704.493935
4            5      No      No   785.655883  38463.495879
5            6      No     Yes   919.588530   7491.558572
6            7      No      No   825.513331  24905.226578
7            8      No     Yes   808.667504  17600.451344
8            9      No      No  1161.057854  37468.529288
9           10      No      No     0.000000  29275.268293
10          11      No     Yes     0.000000  21871.073089
11          12      No     Yes  1220.583753  13268.562221
12          13      No      No   237.045114  28251.695345
13          14      No      No   606.742343  44994.555849
14          15      No      No  1112.968401  23810.174050
15          16      No      No   286.232560  45042.413036
16          17

In [51]:
#eliminacion de nulos
data = data.dropna()

#codificacion

data['student'] = data['student'].map({'No': 0, 'Yes': 1})
data['default'] = data['default'].map({'No': 0, 'Yes': 1})

print(data.head())

from sklearn.preprocessing import StandardScaler

# Columnas numéricas
numerical_columns = ['balance', 'income']

# Inicializar el escalador
scaler = StandardScaler()

# Escalar las características numéricas
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

print(data.head())


   Unnamed: 0  default  student      balance        income
0           1        0        0   729.526495  44361.625074
1           2        0        1   817.180407  12106.134700
2           3        0        0  1073.549164  31767.138947
3           4        0        0   529.250605  35704.493935
4           5        0        0   785.655883  38463.495879
   Unnamed: 0  default  student   balance    income
0           1        0        0 -0.218835  0.813187
1           2        0        1 -0.037616 -1.605496
2           3        0        0  0.492410 -0.131212
3           4        0        0 -0.632893  0.164031
4           5        0        0 -0.102791  0.370915


In [52]:
from sklearn.model_selection import train_test_split
# Definir variables
X = data[['balance', 'income', 'student']]  # Características
y = data['default']  # objetivo

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)


In [60]:
#Creación del Modelo:
from sklearn.linear_model import LogisticRegression

# Crear un modelo de regresión logística
model = LogisticRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit (X_train, y_train)

#predicciones
y_pred = model.predict(X_test)

#probabilidades
y_pred_prob = model.predict_proba(X_test)

#matriz de confusion
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

#informe de clasificacion
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

#precisión
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

#exactitud
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

#sensibilidad
recall = recall_score(y_test, y_pred)
print("Sensibilidad:", recall)

#puntuación F1
f1 = f1_score(y_test, y_pred)
print("Puntuación F1:", f1)

#especificidad
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print("Especificidad:", specificity)

#puntuación de precisión general.
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='weighted')
print("Puntuación de precisión general:", precision)



              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1930
           1       0.81      0.31      0.45        70

    accuracy                           0.97      2000
   macro avg       0.90      0.66      0.72      2000
weighted avg       0.97      0.97      0.97      2000

Precision: 0.8148148148148148
Accuracy: 0.9735
Sensibilidad: 0.3142857142857143
Puntuación F1: 0.4536082474226804
Especificidad: 0.9974093264248705
Puntuación de precisión general: 0.9700415798464456


In [59]:
#statsmodel
import statsmodels.api as sm

# Agregar constante para la regresión logística
X_train_const = sm.add_constant(X_train)

# Entrenar modelo con statsmodels
model_statsmodels = sm.Logit(y_train, X_train_const)
result = model_statsmodels.fit()

# Mostrar resumen del modelo
print(result.summary())

# Predicciones en el conjunto de prueba
X_test_const = sm.add_constant(X_test)
y_pred_prob = result.predict(X_test_const)
y_pred_statsmodels = (y_pred_prob >= 0.5).astype(int)

# Evaluación del modelo
accuracy_stats = accuracy_score(y_test, y_pred_statsmodels)
precision_stats = precision_score(y_test, y_pred_statsmodels)
recall_stats = recall_score(y_test, y_pred_statsmodels)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_statsmodels).ravel()
specificity_stats = tn / (tn + fp)

# Mostrar resultados
print(f"Accuracy: {accuracy_stats:.2f}")
print(f"Precision: {precision_stats:.2f}")
print(f"Recall (Sensibilidad): {recall_stats:.2f}")
print(f"Specificity (Especificidad): {specificity_stats:.2f}")

Optimization terminated successfully.
         Current function value: 0.078622
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7996
Method:                           MLE   Df Model:                            3
Date:                Sat, 01 Mar 2025   Pseudo R-squ.:                  0.4563
Time:                        04:50:47   Log-Likelihood:                -628.97
converged:                       True   LL-Null:                       -1156.8
Covariance Type:            nonrobust   LLR p-value:                1.544e-228
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.9854      0.217    -27.614      0.000      -6.410      -5.561
balance        2.7522      0

In [63]:
#Comparación
print("Accuracy Sklearn:", accuracy)
print(f"Accuracy statsmodels: {accuracy_stats:.2f}")

print("Precision Sklearn:", precision)
print(f"Precision statsmodels: {precision_stats:.2f}")

print("Sensibilidad Sklearn:", recall)
print(f"Sensibilidad statsmodels: {recall_stats:.2f}")

print("Puntuación F1 Sklearn:", f1)
print(f"Puntuación F1 statsmodels: {f1:.2f}")

print("Especificidad Sklearn:", specificity)
print(f"Especificidad statsmodels: {specificity:.2f}")

print("Puntuación de precisión general Sklearn:", precision)
print(f"Puntuación de precisión general statsmodels: {precision_stats:.2f}")


print ("Los resultados son similares")

Accuracy Sklearn: 0.9735
Accuracy statsmodels: 0.97
Precision Sklearn: 0.9700415798464456
Precision statsmodels: 0.81
Sensibilidad Sklearn: 0.3142857142857143
Sensibilidad statsmodels: 0.31
Puntuación F1 Sklearn: 0.4536082474226804
Puntuación F1 statsmodels: 0.45
Especificidad Sklearn: 0.9974093264248705
Especificidad statsmodels: 1.00
Puntuación de precisión general Sklearn: 0.9700415798464456
Puntuación de precisión general statsmodels: 0.81
Los resultados son similares
